In [ ]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

In [ ]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

In [ ]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

In [ ]:
import torch

# Set the device for training and querying the model.
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'The selected device is: "{DEVICE}"')

The selected device is: "cuda"


# Loading the Data

In [ ]:
import os

BASE_DATA_DIR = os.path.join('.', 'data', 'metr-la')

In [ ]:
import pickle
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [ ]:
from src.spatial_temporal_gnn.model import SpatialTemporalGNN
from src.data.data_extraction import get_adjacency_matrix

# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(BASE_DATA_DIR, 'raw', 'adj_mx_metr_la.pkl'))

# Get the header of the adjacency matrix, the node indices and the
# matrix itself.
header, node_ids_dict, adj_matrix = adj_matrix_structure

# Get the STGNN and load the checkpoints.
spatial_temporal_gnn = SpatialTemporalGNN(9, 1, 12, 12, adj_matrix, DEVICE, 64)

stgnn_checkpoints_path = os.path.join('..', 'models', 'checkpoints',
                                      'st_gnn_metr_la.pth')

stgnn_checkpoints = torch.load(stgnn_checkpoints_path)
spatial_temporal_gnn.load_state_dict(stgnn_checkpoints['model_state_dict'])

# Set the STGNN in evaluation mode.
spatial_temporal_gnn.eval();

In [ ]:
from src.data.data_extraction import get_locations_dataframe

# Get the dataframe containing the latitude and longitude of each sensor.
locations_df = get_locations_dataframe(
    os.path.join(BASE_DATA_DIR, 'raw', 'graph_sensor_locations_metr_la.csv'),
    has_header=True)

In [ ]:
# Get the node positions dictionary.
node_pos_dict = { i: id for id, i in node_ids_dict.items() }

In [ ]:
import pickle

# Get the data scaler.
with open(os.path.join(BASE_DATA_DIR, 'processed', 'scaler.pkl'), 'rb') as f:
    scaler = pickle.load(f)

In [ ]:
import os
import numpy as np

# Get the data and the values predicted by the STGNN.
x_train = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_train.npy'))
y_train = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_train.npy'))
x_train_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_train_time.npy'))
y_train_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_train_time.npy'))

x_val = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_val.npy'))
y_val = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_val.npy'))
x_val_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_val_time.npy'))
y_val_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_val_time.npy'))

x_test = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_test.npy'))
y_test = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_test.npy'))
x_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'x_test_time.npy'))
y_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explainable', 'y_test_time.npy'))

In [ ]:
from src.data.data_processing import get_distance_matrix

if not os.path.exists(
    os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy')):
    # Build the distance matrix between the nodes.
    distance_matrix = get_distance_matrix(
        adj_matrix,
        locations_df,
        node_pos_dict)

    # Save the distance matrix.
    np.save(os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy'),
            distance_matrix)

else:
    # Load the distance matrix.
    distance_matrix = np.load(
        os.path.join(BASE_DATA_DIR, 'processed', 'distance_matrix.npy'))

In [ ]:
from src.explanation.monte_carlo.explanation import apply_grid_search

apply_grid_search(
    x_train[::10],
    y_train[::10],
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts_list=[30, 50],
    explanation_size_factor_list=[2, 3],
    cut_size_factor_list=[2],
    exploration_weight_list=[5, 10, 20],
    remove_value_list=[0., -10.])

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 30 remove_value: 0.0
[100/100] - 605s - MAE: { severe_congestion 3.36 -congestion 1.61 -free_flow 0.889 - total: 1.91 } - RMSE: { severe_congestion 4 -congestion 2.01 -free_flow 1.09 - total: 2.31 } - MAPE: { severe_congestion 15% -congestion 3.39% -free_flow 1.36% - total: 6.5% } - Average time: 6.05s 

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 30 remove_value: -10.0
[100/100] - 603s - MAE: { severe_congestion 4.15 -congestion 1.65 -free_flow 1.16 - total: 2.28 } - RMSE: { severe_congestion 5.31 -congestion 2.05 -free_flow 1.43 - total: 2.88 } - MAPE: { severe_congestion 20% -congestion 3.49% -free_flow 1.78% - total: 8.4% } - Average time: 6.03s 

Testing: cut_size_factor: 2 explanation_size_factor: 2 exploration_weight: 5 n_rollouts: 50 remove_value: 0.0
[100/100] - 924s - MAE: { severe_congestion 3.25 -congestion 1.4 -free_flow 0.71 - total: 1.76 }

In [ ]:
CUT_SIZE_FACTOR = 2
EXPLANATION_SIZE_FACTOR = 2
EXPLORATION_WEIGHT = 20
N_ROLLOUTS = 50

In [ ]:
import os
import numpy as np

EXPLAINED_DATA_DIR = os.path.join(BASE_DATA_DIR, 'explained')
os.makedirs(EXPLAINED_DATA_DIR, exist_ok=True)


In [ ]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the training set...')
x_train_explained, y_train_explained, train_scores = get_all_explanations(
    x_train,
    y_train,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_train.npy'), x_train_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_train.npy'), y_train_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'train_scores.npy'), train_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_train_time.npy'), x_train_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_train_time.npy'), y_train_time)


Computing the explanations for the training set...
[999/999] - 8995s - MAE: { severe_congestion 3.14 -congestion 1.49 -free_flow 0.673 - total: 1.76 } - RMSE: { severe_congestion 3.83 -congestion 1.85 -free_flow 0.842 - total: 2.17 } - MAPE: { severe_congestion 13.8% -congestion 3.02% -free_flow 1.03% - total: 5.95% } - Average time: 9s 


In [ ]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the validation set...')
x_val_explained, y_val_explained, val_scores = get_all_explanations(
    x_val,
    y_val,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_val.npy'), x_val_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_val.npy'), y_val_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'val_scores.npy'), val_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_val_time.npy'), x_val_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_val_time.npy'), y_val_time)

Computing the explanations for the validation set...
[198/198] - 1686s - MAE: { severe_congestion 3.75 -congestion 1.56 -free_flow 0.639 - total: 1.94 } - RMSE: { severe_congestion 4.44 -congestion 1.93 -free_flow 0.809 - total: 2.34 } - MAPE: { severe_congestion 17.7% -congestion 3.09% -free_flow 0.974% - total: 7.08% } - Average time: 8.51s 


In [18]:
from src.explanation.monte_carlo.explanation import get_all_explanations


print('Computing the explanations for the test set...')
x_test_explained, y_test_explained, test_scores = get_all_explanations(
    x_test,
    y_test,
    distance_matrix,
    spatial_temporal_gnn,
    scaler,
    n_rollouts=N_ROLLOUTS,
    explanation_size_factor=EXPLANATION_SIZE_FACTOR,
    cut_size_factor=CUT_SIZE_FACTOR,
    exploration_weight=EXPLORATION_WEIGHT,
    remove_value=0.,
    divide_by_traffic_cluster_kind=True)

# Save the explained data.
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_test.npy'), x_test_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_test.npy'), y_test_explained)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'test_scores.npy'), test_scores)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'x_test_time.npy'), x_test_time)
np.save(os.path.join(EXPLAINED_DATA_DIR, 'y_test_time.npy'), y_test_time)

Computing the explanations for the test set...
[300/300] - 2710s - MAE: { severe_congestion 3.27 -congestion 1.7 -free_flow 0.75 - total: 1.89 } - RMSE: { severe_congestion 3.94 -congestion 2.1 -free_flow 0.931 - total: 2.3 } - MAPE: { severe_congestion 14.2% -congestion 3.58% -free_flow 1.15% - total: 6.25% } - Average time: 9.03s 
